## Data Validation with Voluptuous (Schema Definitions)

In this notebook, we'll use [Voluptuous](https://github.com/alecthomas/voluptuous) to define schemas for our data. We can then use schema checking at different points in our cleanup to ensure we meet criteria. We can then use schema validation exceptions to either mark, set aside or remove unclean / invalid data. 

In [1]:
import logging
import pandas as pd
from datetime import datetime
from voluptuous import Schema, Required, Range, All, ALLOW_EXTRA
from voluptuous.error import MultipleInvalid, Invalid

In [2]:
logger = logging.getLogger(0)
logger.setLevel(logging.WARNING)

In [3]:
sales = pd.read_csv('../data/sales_data.csv')

### Data Quality Check

In [4]:
sales.head()

,timestamp,city,store_id,sale_number,sale_amount,associate
0,2017-02-19T17:00:00,Stephanieport,11,2162,247.0,Jenna White
1,2017-02-19T22:00:00,Gutierreztown,11,754,1586.0,Laura Massey
2,2017-02-20T01:00:00,Colemanside,3,2858,631.0,Jacqueline Benson
3,2017-02-20T08:00:00,Harriston,1,1080,-161.0,Tina Martin
4,2017-02-20T13:00:00,Lake Fernandoton,1,358,1414.0,David Khan


In [5]:
sales.dtypes

timestamp       object
city            object
store_id         int64
sale_number      int64
sale_amount    float64
associate       object
dtype: object

## Defining our first schema

In [6]:
schema = Schema({
    # sale amount is required, and for all of them I expect them to be floats in the rand min to max
    Required('sale_amount'): All(float, Range(min=2.50, max=1450.99)),
    #
}, extra=ALLOW_EXTRA)

In [7]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', s_id, sale['sale_amount'], e)
        error_count += 1

In [8]:
error_count

110

In [9]:
sales.shape

(369, 6)

### Questions we might want to answer:
- Do we have an improperly defined schema?
- Are negative values possibly returns or falsely marked? (data entry proceedures)
- Are higher values combined purchases or special sales? (or potentially fraud?)
- What should we do with our schema and our failing data points?

### Adding a custom Validation Case

In [10]:
def ValidDate(fmt='%Y-%m-%dT%H:%M:%S'):
    return lambda v: datetime.strptime(v, fmt)

In [11]:
schema = Schema({
    Required('timestamp'): All(ValidDate()),
}, extra=ALLOW_EXTRA)

In [12]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', s_id, sale['timestamp'], e)
        error_count += 1

In [13]:
error_count

0

## So we have valid date structures, what about actual valid dates?

In [14]:
def ValidDate(fmt='%Y-%m-%dT%H:%M:%S'):
    def validation_func(v):
        try:
            assert datetime.strptime(v, fmt) <= datetime.now()
        except AssertionError:
            raise Invalid('date is in the future! %s' % v)
    return validation_func

In [15]:
schema = Schema({
    Required('timestamp'): All(ValidDate()),
}, extra=ALLOW_EXTRA)

In [16]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', s_id, sale['timestamp'], e)
        error_count += 1

In [17]:
error_count

0

## Exercise: what are some possible reasons for future dates? What should we do with the data and schema?